# Assignment 8.2

## Lincoln Brown
## DSC530-T301
## Dr. Parajulee

In [3]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import random

import thinkstats2
import thinkplot
import statsmodels.formula.api as smf

In [10]:
import first
import nsfg

live, firsts, others = first.MakeFrames()

## Exercise 11-1
Suppose one of your co-workers is expecting a baby an dyou are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variable sthat are known before the birth, and likely to be available to the people in the pool.


First we will limit live to prglngth greater than 30 wks

In [51]:
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [67]:
def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ agepreg +'+ name
            #formula = formula.encode('ascii')

            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError):
            continue
        except patsy.PatsyError:
            raise ValueError(MESSAGE)

        variables.append((results.rsquared, name))

    return variables

In [68]:
variables = GoMining(join)

In [63]:
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except KeyError:
            print(name, r2)

In [64]:
MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062434116139252 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120192
birthwgt_lb 0.11977307804917103 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372542204582791 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592313 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022053775796472164 PRGLNGTH IMPUTATION FLAG
canhaver 0.006050495268196787 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
datcon01_i 0.005817755299873495 DATCON01 IMPUTATION FLAG
con1mar1_i 0.005546376136239095 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.004577565785535143 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
mar1con1_i 0.003150802253861107 MAR1CON1 IMPUTATION FLAG
anynurse 0.0024520248837108793 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.002369183944671338 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.0022493894337984877 BC-1 HOW PREGNANCY ENDED - 1ST ME

Trying to use data mining to uncover significant predictors did not turn out very well. 
There are too many predictors in the dct files to really return anything of use.

It is more important to focus on predictors that are available to the people in the pool.

These types of things are, is this her first child? 

What is her age?

What is her race?

Allen includes the birthord==1 and nbrnaliv>1 in his formula. At first I didn't understand why he did this, but it makes sense to include both to account for all possibilities. 

If it is the first baby being born it will be birthord==1. However, if she has had other children before, this will be accounted for by nbrnaliv>1.



In [83]:
formula = ('prglngth ~ agepreg + birthord==1 + race==2 + nbrnaliv>1')
model = smf.ols(formula, data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     25.72
Date:                Sun, 01 Aug 2021   Prob (F-statistic):           3.16e-21
Time:                        15:05:04   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8879   BIC:                         3.654e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7357      0.105    367.466      0.000      38.529      38.942
birthord == 1[T.True]     0.1052      0.043      2.473      0.013       0.022       0.189
race == 2[T.True]         0.1369      0.043      3.202      0.001       0.053       0.221
nbrnaliv > 1[T.True]     -1.4956      0.165     -9.089      0.000      -1.818      -1.173
agepreg                   0.0010      0.004      0.264      0.792      -0.007       0.009
==============================================================================
Omnibus:                     1587.420   Durbin-Watson:                   1.620
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6159.875
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         210.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

From these results we can see that agepreg didn't make much of a difference at all and could be left out of the model. 

nbrnaliv has a negative coefficient which indicates a negative relationship and birthord, race, and nbrnaliv all have significant p values. 

The race==2 indicates that the coworker is white and may need to be changed where applicable. 


## Exercise 11-3
If the quantity you want to predict is a count you can use Poisson regression, which is implemented in StatsModels with a function called poisson. It works the same way as ols and logit. As an exercise, let's use it to predict how many children a woman has born; in the NSFG dataset, this variable is called numbabes.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?


In [122]:
import regression
my_join = regression.JoinFemResp(live)
my_formula = 'numbabes ~ age_r + C(race) + totincr + educat'
my_model = smf.poisson(my_formula, data=my_join)
my_results = my_model.fit()
my_results.summary()

Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Sun, 01 Aug 2021   Pseudo R-squ.:                 0.03109
Time:                        16:11:21   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
age_r            0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

In [123]:
# Now I want to compare the difference between the models in the book that have the additional corrections for age
join = regression.JoinFemResp(live)
#join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2
join['age3'] = join.age_r**3

In [125]:
formula = 'numbabes ~ age_r + age2 + age3 + C(race) + totincr + educat'
new_model = smf.poisson(formula, data=join)
results = new_model.fit()
results.summary()

         Current function value: 749175786307.141235
         Iterations: 35


/home/x/snap/jupyter/common/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8876
Method:                           MLE   Df Model:                            7
Date:                Sun, 01 Aug 2021   Pseudo R-squ.:              -4.303e+11
Time:                        16:11:43   Log-Likelihood:            -6.6557e+15
converged:                      False   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                     1.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -36.3136        nan        nan        nan         nan         nan
C(race)[T.2]    -0.0013   2.81e-08  -4.65e+04      0.000      -0.001      -0.001
C(race)[T.3]     0.0017   5.95e-08   2.78e+04      0.000       0.002       0.002
age_r           -0.1943        nan        nan        nan         nan         nan
age2            -0.0082        nan        nan        nan         nan         nan
age3             0.0011        nan        nan        nan         nan         nan
totincr          0.0010    3.8e-09   2.63e+05      0.000       0.001       0.001
educat           0.0010   5.44e-09   1.84e+05      0.000       0.001       0.001
================================================================================
"""

In [129]:
import pandas
columns = ["age_r", "race", "totincr", "educat"]
guess = pandas.DataFrame([[35, 1, 14, 16]], columns=columns)
my_results.predict(guess)

0    2.342182
dtype: float64

The model predicts that she will most likely have two children. 

## Exercise 11-4
If the quantity you want to predict is categorical you can use multinomial logistic regression, which is implemented in StatsModels with a function called mnlogit. As an exercise, let's use it to guess whether a woman is married; in the NSFG dataset, marital status is encoded in a variable called rmarital. 

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitation, etc?

In [131]:
formula = 'rmarital ~ age_r + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=my_join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Sun, 01 Aug 2021   Pseudo R-squ.:                  0.1655
Time:                        16:14:40   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
age_r           -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
age_r            0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
age_r            0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
age_r            0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

In [132]:
columns = ['age_r', 'race', 'totincr', 'educat']
guess = pandas.DataFrame([[25, 2, 11, 12]], columns = columns)
results.predict(guess)

,0,1,2,3,4,5
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931


A 25 year old, female, white high school graduate with an annual household income of about $45,000 has a 75 percent chance of being married. 
13 percent chance of cohabitating with opposite sex partner.
Less than a one percent chance of being a widow.
A two percent chance of being separated for reasons of marital discord.
A six percent chance that she has never been married. 
